In [1]:
import numpy as np
import sys
import os
import random
import somim_module

In [ ]:
# Randomization Functions
def randomize_prob(num_states=2):
    p = np.random.rand(num_states)
    p /= np.sum(p)

    return p

def randomize_density(n=2):
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    rho = A @ A.conj().T
    rho /= np.trace(rho)

    return rho

def generate_ensemble(n, J, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    rho = [randomize_density(n) for _ in range(J)]
    p = randomize_prob(J)

    return p, rho

In [3]:
n = 2             # Dimension of Hilbert space (e.g., n=2 for qubit, n=4 for two-qubit system)
J = 2             # Number of quantum states in ensemble
K_max = n**2      # Max number of POVM elements

In [4]:
seed = 42
# seed = None

In [5]:
print(f"Generating random ensemble (N={n}, J={J})...")
p, rho_list = generate_ensemble(n, J, seed)
print(f"Ensemble:")
for j in range(J):  
    print(f"State {j+1} with probability {p[j]:.6f}:\n{rho_list[j]}\n")     
# Convert to C++ tensor format
E_np = np.zeros((n, n, J), dtype=np.complex128)
for j in range(J):
    E_np[:, :, j] = p[j] * rho_list[j]

Generating random ensemble (N=2, J=2)...
Ensemble:
State 1 with probability 0.814342:
[[ 0.06058752+0.j         -0.07072673-0.19145922j]
 [-0.07072673+0.19145922j  0.93941248+0.j        ]]

State 2 with probability 0.185658:
[[0.53209671+0.j         0.07833537+0.03446115j]
 [0.07833537-0.03446115j 0.46790329+0.j        ]]



In [6]:
print("Initializing SOMIM C++ Solver...")
solver = somim_module.SOMIM(n, J, K_max)
solver.setDirectGrad(50)
solver.setTolerance(1e-10)
print("Optimizing...")
solver.set_ensemble_from_numpy(E_np)
accessible_info = solver.getMI()
povm = solver.get_povm_as_numpy()
print("Optimization complete.")

Initializing SOMIM C++ Solver...
Optimizing...
Optimization complete.


In [7]:
print(f"\nResult found!")
print(f"Accessible Information: {accessible_info:.8f} bits")
print(f"Optimal POVM has {povm.shape[2]} elements.")
print(f"POVM Elements:")
for k in range(povm.shape[2]):  
    print(f"Element {k+1}:\n{povm[:, :, k]}\n") 


Result found!
Accessible Information: 0.23908172 bits
Optimal POVM has 2 elements.
POVM Elements:
Element 1:
[[0.94898865+0.j         0.09024167+0.20066299j]
 [0.09024167-0.20066299j 0.05101135+0.j        ]]

Element 2:
[[ 0.05101135+0.j         -0.09024167-0.20066299j]
 [-0.09024167+0.20066299j  0.94898865+0.j        ]]

